# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121780e+02     1.411379e+00
 * time: 0.6467881202697754
     1     1.000811e+01     8.957799e-01
 * time: 1.7518610954284668
     2    -1.196437e+01     9.557214e-01
 * time: 2.3314590454101562
     3    -3.417866e+01     8.123438e-01
 * time: 3.1426279544830322
     4    -4.763700e+01     5.790681e-01
 * time: 3.96492600440979
     5    -5.716639e+01     2.486381e-01
 * time: 4.77888298034668
     6    -5.996395e+01     1.539568e-01
 * time: 5.441251993179321
     7    -6.098977e+01     4.802487e-02
 * time: 6.00477409362793
     8    -6.138587e+01     5.916418e-02
 * time: 6.559446096420288
     9    -6.161624e+01     4.113447e-02
 * time: 7.108334064483643
    10    -6.180976e+01     2.565887e-02
 * time: 8.025820970535278
    11    -6.196242e+01     2.276160e-02
 * time: 8.571614980697632
    12    -6.204547e+01     1.719422e-02
 * time: 9.12139105796814
    13    -6.210556e+01     1.347280e-02
 * time: 9.690142154693604
    14 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671052
    AtomicLocal         -18.8557599
    AtomicNonlocal      14.8522601
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485343 
    Xc                  -19.3336809

    total               -62.261666459695
